# Exporting Data to Pardiso (Modular Backend) and Calculating Band Structure

This tutorial demonstrates the **new modular backend** for Pardiso integration in DeePTB.
It uses:
1. `to_pardiso_new` method: Exports data in an optimized JSON (Optimade-like) + HDF5 format.
2. Modular Julia Backend (`main.jl`): A structured, extensible Julia codebase for calculations.

## Objectives
1. Initialize a `TBSystem`.
2. Export data using the optimized `to_pardiso_new`.
3. Run the modular Julia script (`main.jl`) to calculate band structure.
4. Visualize results.

## 1. Setup Paths

Define paths to model, structure, and the NEW output directory.

In [ ]:
import os
import shutil
import sys
import json
import subprocess

# Ensure dptb is in path
sys.path.insert(0, os.path.dirname(os.path.dirname(os.getcwd())))
from dptb.postprocess.unified.system import TBSystem

root_dir = os.path.dirname(os.path.abspath("__file__")) 
# Note: In Jupyter __file__ might not exist, using current dir if needed
if not os.path.exists(root_dir) or root_dir == '':
    root_dir = os.getcwd()

model_path = os.path.join(root_dir,  "nnsk.iter_ovp0.000.pth")
struct_path = os.path.join(root_dir, "min.vasp")
output_dir = os.path.join(root_dir, "output_new_notebook")
julia_out_dir = os.path.join(output_dir, "results")

print(f"Root Dir: {root_dir}")
print(f"Output Dir: {output_dir}")

## 2. Initialize TBSystem

Load structure and model. Note that `TBSystem` now directly handles the structure without extra wrappers.

In [ ]:
tbsys = TBSystem(data=struct_path, calculator=model_path)
print("TBSystem initialized successfully.")
print(f"System Info: {tbsys.atoms}")

## 3. Export Data (New Method)

We use `to_pardiso_new` which generates:
- `structure.json`: Minimized, readable structure & basis info.
- `predicted_hamiltonians.h5`: Sparse Hamiltonian blocks.
- `predicted_overlaps.h5`: Sparse Overlap blocks.

In [ ]:
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
os.makedirs(output_dir, exist_ok=True)

# Call the new export method
tbsys.to_pardiso_new(output_dir=output_dir)

print(f"Data exported to {output_dir}")
print("Generated files:")
for f in sorted(os.listdir(output_dir)):
    print(f" - {f}")

## 4. Run Modular Julia Backend

We call `dptb/postprocess/julia/main.jl`. This script parses the `structure.json` automatically.

In [ ]:
parent_path = os.path.dirname(os.path.dirname(os.getcwd()))
julia_script = os.path.join(parent_path, "dptb/postprocess/julia/main.jl")
config_path = os.path.join(root_dir, "band.json")

if not os.path.exists(julia_out_dir):
    os.makedirs(julia_out_dir)

cmd = [
    "julia",
    julia_script,
    "--input_dir", output_dir,
    "--output_dir", julia_out_dir,
    "--config", config_path
]

print(f"Running Command: {' '.join(cmd)}")

try:
    result = subprocess.run(cmd, capture_output=True, text=True, check=True)
    print("\nJulia Output:")
    print(result.stdout)
    print("Julia Execution Successful!")
except subprocess.CalledProcessError as e:
    print("Julia script failed with error:")
    print(e.stdout)
    print(e.stderr)

## 5. Visualizing Results

Load `bandstructure.h5` generated by the Julia backend.

In [ ]:
expected_h5 = os.path.join(julia_out_dir, "bandstructure.h5")
import h5py
import matplotlib.pyplot as plt

print(f"Loading results from {expected_h5}...")
with h5py.File(expected_h5, 'r') as f:
    evals = f['eigenvalues'][:]
    x = f['xlist'][:]
    e_fermi = f['E_fermi'][()]
    high_sym_kpoints = f['high_sym_kpoints'][:]
    labels_bytes = f['labels'][:]
    
# Decode labels from bytes to string
try:
    labels = [l.decode('utf-8') for l in labels_bytes]
except AttributeError:
    # If already strings
    labels = list(labels_bytes)


print(f"Successfully loaded bandstructure.h5")
print(f"Eigenvalues shape: {evals.shape}")

plt.figure(figsize=(6,5))

evals_shifted = evals - e_fermi
for i in range(evals.shape[1]): # Loop over bands
    plt.scatter(x, evals_shifted[:, i], s=1, c='blue') 
    
plt.xticks(high_sym_kpoints, labels)
plt.xlim(0,x[-1])
plt.ylim(-2,2)
plt.ylabel('E - EF (eV)')
plt.title('Band Structure (Modular Julia Backend)')
plt.grid(True, alpha=0.3)
plt.show()